In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
# Download the dataset
!wget https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip

--2020-08-31 08:41:43--  https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip
Resolving s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)... 52.219.112.16
Connecting to s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)|52.219.112.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1132023110 (1.1G) [application/zip]
Saving to: ‘dogImages.zip’

dogImages.zip       100%[===================>]   1.05G  48.5MB/s    in 21s     

2020-08-31 08:42:05 (50.3 MB/s) - ‘dogImages.zip’ saved [1132023110/1132023110]



In [ ]:
# Extract dataset
import zipfile
with zipfile.ZipFile('dogImages.zip', 'r') as zip_ref:
    zip_ref.extractall('')

# Rename dataset
!mv dogImages dataset

In [ ]:
# Fix pour Truncated File OS Error
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# 1 - Construction du CNN
from keras.models import Sequential
from keras.models import load_model

# Initialiser CNN
classifier = Sequential()

# Charger le modèle
classifier = load_model('/content/drive/My Drive/CNN-for-dog-breed/checkpoint-02-0.74.hdf5')
classifier.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 2048)              20861480  
_________________________________________________________________
batch_normalization_15 (Batc (None, 2048)              8192      
_________________________________________________________________
dropout_16 (Dropout)         (None, 2048)              0         
_________________________________________________________________
dense_16 (Dense)             (None, 1024)              2098176   
_________________________________________________________________
dropout_17 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_17 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
dropout_18 (Dropout)         (None, 1024)             

In [ ]:
NB_BREEDS = 133
INPUT_SIZE = 512
BATCH_SIZE = 16
NB_TRAIN = 6680
NB_VALID = 835
NB_EPOCH = 20

In [ ]:
from keras.applications import Xception

xception = Xception(weights='imagenet', 
                                 include_top = False, 
                                 input_shape = (INPUT_SIZE, INPUT_SIZE,3),
                                 pooling = 'max')

83689472/83683744 [==============================] - 1s 0us/step


In [ ]:
classifier.add(xception)
# Rendre les couches du réseau XCeption non-entrainable (puisque déjà 
# entrainés)
for layer in classifier.layers:
    layer.trainable = False

from keras.layers import Dense, Dropout, BatchNormalization
classifier.add(BatchNormalization())
# Fully connected
classifier.add(Dropout(0.1))
classifier.add(Dense(units=1024, activation="relu"))
classifier.add(Dropout(0.1))
classifier.add(Dense(units=1024, activation="relu"))
classifier.add(Dropout(0.1))
classifier.add(Dense(units=1024, activation="relu"))
classifier.add(Dropout(0.1))
classifier.add(Dense(units=1024, activation="relu"))
classifier.add(Dropout(0.1))
classifier.add(Dense(units=1024, activation="relu"))
classifier.add(Dropout(0.1))

# Predictions
classifier.add(Dense(units=NB_BREEDS, activation="softmax"))

# Compilation
from keras.optimizers import RMSprop
classifier.summary()
classifier.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 2048)              20861480  
_________________________________________________________________
batch_normalization_4 (Batch (None, 2048)              8192      
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              2098176   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0

In [ ]:
# Entraînement sur les images !
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)
training_set = train_datagen.flow_from_directory(
        'dataset/train',
        target_size=(INPUT_SIZE,INPUT_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical')
test_set = test_datagen.flow_from_directory(
        'dataset/valid',
        target_size=(INPUT_SIZE,INPUT_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical')

Found 6680 images belonging to 133 classes.
Found 835 images belonging to 133 classes.


In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

''

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
callback = EarlyStopping(monitor='loss', patience=3)
checkpoint = ModelCheckpoint("/content/drive/My Drive/CNN-for-dog-breed/checkpoint-bis-{epoch:02d}-{val_accuracy:.2f}.hdf5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

history = classifier.fit(
        training_set,
        steps_per_epoch=NB_TRAIN // BATCH_SIZE,
        epochs=NB_EPOCH,
        validation_data=test_set,
        validation_steps=NB_VALID // BATCH_SIZE,
        callbacks=[callback, checkpoint])

classifier.save("/content/drive/My Drive/CNN-for-dog-breed/final-xception.h5")

import matplotlib.pyplot as plt
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

Epoch 1/20
417/417 [==============================] - ETA: 0s - loss: 1.0430 - accuracy: 0.7383 
Epoch 00001: val_accuracy improved from -inf to 0.79688, saving model to /content/drive/My Drive/Perso/IA/CNN-for-dog-breed/checkpoint-bis-01-0.80.hdf5
417/417 [==============================] - 7530s 18s/step - loss: 1.0430 - accuracy: 0.7383 - val_loss: 0.9334 - val_accuracy: 0.7969
Epoch 2/20
417/417 [==============================] - ETA: 0s - loss: 0.8716 - accuracy: 0.7746 
Epoch 00002: val_accuracy improved from 0.79688 to 0.81611, saving model to /content/drive/My Drive/Perso/IA/CNN-for-dog-breed/checkpoint-bis-02-0.82.hdf5
417/417 [==============================] - 7546s 18s/step - loss: 0.8716 - accuracy: 0.7746 - val_loss: 0.7388 - val_accuracy: 0.8161
Epoch 3/20
 37/417 [=>............................] - ETA: 1:39:58 - loss: 0.6534 - accuracy: 0.8091